### 不知所云, sorry

In [1]:
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import (
    PydanticOutputParser,
    OutputFixingParser,
    RetryOutputParser,
)
from pydantic import BaseModel, Field, validator
from typing import List

In [2]:
template = """Based on the user question, provide an Action and Action Input for what step should be taken.
{format_instructions}
Question: {query}
Response:"""

In [3]:
print(template)

Based on the user question, provide an Action and Action Input for what step should be taken.
{format_instructions}
Question: {query}
Response:


In [4]:
class Action(BaseModel):
    action: str = Field(description="action to take")
    action_input: str = Field(description="input to the action")

### 要素1: OutputParser

In [5]:
parser = PydanticOutputParser(pydantic_object=Action)

In [20]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"action": {"title": "Action", "description": "action to take", "type": "string"}, "action_input": {"title": "Action Input", "description": "input to the action", "type": "string"}}, "required": ["action", "action_input"]}
```


### 要素2: Prompt

In [7]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [8]:
prompt_value = prompt.format_prompt(query="who is leo di caprios gf?")

In [24]:
print(prompt_value.text)

Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"action": {"title": "Action", "description": "action to take", "type": "string"}, "action_input": {"title": "Action Input", "description": "input to the action", "type": "string"}}, "required": ["action", "action_input"]}
```
who is leo di caprios gf?



In [10]:
bad_response = '{"action": "search"}'

If we try to parse this response as is, we will get an error:

In [25]:
parser.parse(bad_response)

OutputParserException: Failed to parse Action from completion {"action": "search"}. Got: 1 validation error for Action
action_input
  field required (type=value_error.missing)

If we try to use the OutputFixingParser to fix this error, it will be confused - namely, it doesn't know what to actually put for action input.

In [26]:
fix_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [27]:
fix_parser.parse(bad_response)

Action(action='search', action_input='input')

Instead, we can use the RetryOutputParser, which passes in the prompt (as well as the original output) to try again to get a better response.

In [28]:
from langchain.output_parsers import RetryWithErrorOutputParser

In [29]:
retry_parser = RetryWithErrorOutputParser.from_llm(
    parser=parser, llm=OpenAI(temperature=0)
)

In [30]:
retry_parser.parse_with_prompt(bad_response, prompt_value)

Action(action='search', action_input='who is leo di caprios gf?')